<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/use_cases/text2sql/StructuredLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Use Llama 3 to chat about structured data
This demo shows how to use LangChain's SQLDatabaseChain with Llama 3 to query structured data, the 2023-24 NBA roster info, stored in a SQLite DB, to ask Llama 3 questions about your favorite teams or players.

We start by installing the necessary packages:
- [Replicate](https://replicate.com/) to host the Llama 3 model
- [langchain](https://python.langchain.com/docs/get_started/introduction) provides necessary RAG tools for this demo

**Note** We will be using [Replicate](https://replicate.com/meta/meta-llama-3-8b-instruct) to run the examples here. You will need to first sign in with Replicate with your github account, then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while. You can also use other Llama 3 cloud providers such as [Groq](https://console.groq.com/), [Together](https://api.together.xyz/playground/language/meta-llama/Llama-3-8b-hf), or [Anyscale](https://app.endpoints.anyscale.com/playground) - see Section 2 of the Getting to Know Llama [notebook](https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Getting_to_know_Llama.ipynb) for more information.

If you'd like to run Llama 3 locally for the benefits of privacy, no cost or no rate limit (some Llama 3 hosting providers set limits for free plan of queries or tokens per second or minute), see [Running Llama Locally](https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Running_Llama2_Anywhere/Running_Llama_on_Mac_Windows_Linux.ipynb).

In [1]:
!pip install langchain replicate

In [2]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

 ········


Next we call the Llama 3 8b chat model from Replicate. You can also use Llama 3 70b model by replacing the `model` name with "meta/meta-llama-3-70b-instruct".

In [3]:
from langchain_community.llms import Replicate
llm = Replicate(
    model="meta/meta-llama-3-8b-instruct",
    model_kwargs={"temperature": 0.0, "top_p": 1, "max_new_tokens":500}
)

Next you will need create the `nba_roster.db` file. 

To do this run the following commands while in this folder:
- `python txt2csv.py`  This will convert the `nba.txt` file to `nba_roster.csv`. The `nba.txt` file was created by scraping the NBA roster info from the web.
- Then run `python csv2db.py` to convert `nba_roster.csv` to `nba_roster.db`.

Once you have your `nba_roster.db` ready, we set up the database to be queried by Llama 2 through Langchain's [SQL chains](https://python.langchain.com/docs/use_cases/qa_structured/sql).

In [12]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info=0)

def get_schema():
    return db.get_table_info()

def run_query(query):
    return db.run(query)

In [13]:
get_schema()

'\nCREATE TABLE nba_roster (\n\t"Team" TEXT, \n\t"NAME" TEXT, \n\t"Jersey" TEXT, \n\t"POS" TEXT, \n\t"AGE" INTEGER, \n\t"HT" TEXT, \n\t"WT" TEXT, \n\t"COLLEGE" TEXT, \n\t"SALARY" TEXT\n)'

In [22]:
question = "What team is Klay Thompson on?"
prompt = f"""Based on the table schema below, write a SQL query that would answer the user's question; just return the SQL query and nothing else.

Scheme:
{get_schema()}

Question: {question}

SQL Query:"""

print(prompt)

Based on the table schema below, write a SQL query that would answer the user's question; just return the SQL query and nothing else.

Scheme:

CREATE TABLE nba_roster (
	"Team" TEXT, 
	"NAME" TEXT, 
	"Jersey" TEXT, 
	"POS" TEXT, 
	"AGE" INTEGER, 
	"HT" TEXT, 
	"WT" TEXT, 
	"COLLEGE" TEXT, 
	"SALARY" TEXT
)

Question: What team is Klay Thompson on?

SQL Query:


In [24]:
print(llm.invoke(prompt))

SELECT "Team" FROM nba_roster WHERE "NAME" = 'Klay Thompson';


If you don't have the "just return the SQL query and nothing else" in the prompt above, or even with it but asking Llama 2 which doesn't follow instructions as well as Llama 3, you'll likely get more text other than the SQL query back in the answer.

In [26]:
# how about a follow up question
print(llm.invoke("What's his salary?"))

I'm happy to help! However, I need more information. Who is "his"? Could you please provide more context or clarify who you are referring to?



Since we did not pass any context along with the follow-up to the model it did not know who "his" is and just picked LeBron James.

Let's try to fix the issue that the context (the previous question and answer) was not sent to the model along with the new question.
`SQLDatabaseChain.from_llm` has a parameter "memory" which can be set to a `ConversationBufferMemory` instance, which looks promising.


In [8]:

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
db_chain_memory = SQLDatabaseChain.from_llm(llm, db, memory=memory, 
                                            verbose=True, return_sql=True, 
                                            prompt=PromptTemplate(input_variables=["input", "table_info"], 
                                            template=PROMPT_SUFFIX))

In [9]:
# use the db_chain_memory to run the original question again
question = "Which team is Klay Thompson in"
answer = db_chain_memory.run(question)
print(answer)

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "Which team is Klay Thompson in",
  "history": ""
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Which team is Klay Thompson in\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)",
  "stop": [
    "\nSQLResult:"
  ],
  "history": ""
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:Replicate] Entering LLM run with input:
{
  "prompts": [
    "Only use the following tables:\n\nCREATE TABLE nba_roster (\n\t\"Team\" TEXT, \n\t\"NAME\" TEXT, \n\t\"Jersey\" TEXT, \n\t\"POS\" TEXT, \n\t\"AGE\" INTEGER, \n\t\"HT\" TEXT, \n\t\"WT\" TEXT, \n\t\"COLLEGE\" TEXT, \n\t\"SALARY\" TEXT\n)\n\nQuestion: Which team is Klay Tho

In [10]:
import json

memory.save_context({"input": question},
                    {"output": json.dumps(answer)})
followup = "What's his salary"
followup_answer = db_chain_memory.run(followup)
print(followup_answer)

# "Entering Chain run with input" does show "history" including our question: "Human: Which team is Klay Thompson in"
# but it doesn't get passed to "Entering LLM run with input", so the llm output still is Lebron James.

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "What's his salary",
  "history": "Human: Which team is Klay Thompson in\nAI: Sure thing! Based on the information provided in the `nba_roster` table, Klay Thompson is in the Golden State Warriors. Here's the SQL query to retrieve that information:\n```sql\nSELECT * FROM nba_roster WHERE Team = 'Golden State Warriors';\n```\nThis will return all rows where the `Team` column matches \"Golden State Warriors\", which should only have one row with Klay Thompson's information.\nHuman: Which team is Klay Thompson in\nAI: \"Sure thing! Based on the information provided in the `nba_roster` table, Klay Thompson is in the Golden State Warriors. Here's the SQL query to retrieve that information:\\n```sql\\nSELECT * FROM nba_roster WHERE Team = 'Golden State Warriors';\\n```\\nThis will return all rows where the `Team` column matches \\\"Golden State Warriors\\\", which should only have one row with Klay Thompson'

Looks like we hit a [known issue](https://github.com/langchain-ai/langchain/issues/6918#issuecomment-1632932653) with adding memory to SQLDatabaseChain, even after the [PR](https://github.com/langchain-ai/langchain/pull/7546) that's supposed to fix it was merged. We'll leave the notebook using the experimental SQLDatabaseChain with memory as is. To find a better solution to using SQLDatabaseChain, we can:
1. Look into the SQLDatabaseChain's implementation and the use of ConversationBufferMemory to fix the memory issue;
2. Wait till the SQLDatabaseChain memory issue is really fixed.
We'll update this demo when a good solution is found.